Import thư viện

In [16]:
import torch
import torch.nn as nn
from torchvision import datasets
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.datasets.utils import download_url
import os
import tarfile
import matplotlib.pyplot as plt
import numpy as np


Tải dữ liệu

In [17]:
#dataset_url = "https://s3.amazonaws.com/fast-ai-imageclas/cifar10.tgz"
##with tarfile.open('./cifar10.tgz', 'r:gz') as tar:
    #tar.extractall(path='./data_encoder')

train_ds=datasets.CIFAR10(root='../Deep-Learning-Models/data/cifar10',train= True, transform=transforms.ToTensor())
test_ds=datasets.CIFAR10(root='../Deep-Learning-Models/data/cifar10',train= False, transform=transforms.ToTensor())

batch_size=128
train_loader=DataLoader(train_ds,batch_size=batch_size,shuffle=True)
test_loader=DataLoader(test_ds,batch_size=batch_size,shuffle=False)

Sử dụng GPU

In [18]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [19]:
def ConvLayer(inp, out, ks=3, s=1, p=1):
    return nn.Conv2d(inp,out,kernel_size=ks,stride=s,padding=p)
class CNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.neural_net = nn.Sequential (
            ConvLayer(3, 32), nn.ReLU(),
            ConvLayer(32, 64), nn.ReLU(),
            nn.MaxPool2d(2, 2),
            ConvLayer(64, 128), nn.ReLU(),
            ConvLayer(128, 256), nn.ReLU(),
            nn.MaxPool2d(2, 2),
            ConvLayer(256, 512), nn.ReLU(),
            ConvLayer(512, 1024), nn.ReLU(),
            nn.MaxPool2d(2, 2),
            nn.Flatten(),
            nn.Linear(1024*4*4, 1024), nn.ReLU(),
            nn.Linear(1024, 512), nn.ReLU(),
            nn.Linear(512, 10),
    )
    def forward(self, x):
        return self.neural_net(x)
model = CNN().to(device)

Hàm đánh giá

In [20]:
def accurate_score(out, labels):
    _, preds = torch.max(out, dim=1)
    correct_preds = torch.sum(preds == labels).item()
    total_preds = len(preds)
    accuracy = torch.tensor(correct_preds / total_preds)
    return accuracy

Training model

In [21]:
num_epochs = 15
learning_rate = 0.003
loss_func= nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

for epoch in range(num_epochs):
    total_testing_accuracies = []
    for img, labels in train_loader:
        img =img.to(device)
        labels = labels.to(device)
        pred = model(img)
        loss = loss_func(pred, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
    for test_img, test_labels in test_loader:
        test_img = test_img.to(device)
        test_labels = test_labels.to(device)
        test_pred = model(test_img)
        metrics = accurate_score(test_pred, test_labels)
        total_testing_accuracies.append(metrics)
    print("Epoch Number: ", epoch, "Testing Accuracy Score: ", sum(total_testing_accuracies)/len(total_testing_accuracies))

Epoch Number:  0 Testing Accuracy Score:  tensor(0.4424)
Epoch Number:  1 Testing Accuracy Score:  tensor(0.5547)
Epoch Number:  2 Testing Accuracy Score:  tensor(0.6261)
Epoch Number:  3 Testing Accuracy Score:  tensor(0.6539)
Epoch Number:  4 Testing Accuracy Score:  tensor(0.6610)
Epoch Number:  5 Testing Accuracy Score:  tensor(0.6942)
Epoch Number:  6 Testing Accuracy Score:  tensor(0.6947)
Epoch Number:  7 Testing Accuracy Score:  tensor(0.6967)
Epoch Number:  8 Testing Accuracy Score:  tensor(0.6987)
Epoch Number:  9 Testing Accuracy Score:  tensor(0.6967)
Epoch Number:  10 Testing Accuracy Score:  tensor(0.7048)
Epoch Number:  11 Testing Accuracy Score:  tensor(0.7028)
Epoch Number:  12 Testing Accuracy Score:  tensor(0.6987)
Epoch Number:  13 Testing Accuracy Score:  tensor(0.6919)
Epoch Number:  14 Testing Accuracy Score:  tensor(0.7034)


Lưu mô hình

In [22]:
torch.save(model.state_dict(), 'CNN.pth')

In [23]:
state_dict = torch.load("./CNN.pth")
print(type(state_dict))

<class 'collections.OrderedDict'>


C:\Users\admin\AppData\Local\Temp\ipykernel_6960\4243783952.py:1: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load("./CNN.pth")
